In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

pd.set_option('max_columns', 1000)
pd.set_option('max_rows', 1000)

In [2]:
ORG_PATH = '../data/IBS2_First_2_Years/Organism_values.xlsx'
SPEC_PATH = '../data/IBS2_First_2_Years/Species_weights.xlsx'
TOW_PATH = '../data/IBS2_First_2_Years/Tow_Info.xlsx'

ORG_PATH_3 = '../data/y3/organism_values.xlsx'
SPEC_PATH_3 = '../data/y3/species_weights_201880.xlsx'
TOW_PATH_3 = '../data/y3/tow_info_201880.xlsx'

ORG_PATH_MORE = '../data/organism_values_201880.xlsx'

In [3]:
org_df = pd.read_excel(ORG_PATH, )
org_3_df = pd.read_excel(ORG_PATH_3)
org_more_df = pd.read_excel(ORG_PATH_MORE)

spec_df = pd.read_excel(SPEC_PATH)
spec_3_df = pd.read_excel(SPEC_PATH_3)

tow_df = pd.read_excel(TOW_PATH)
tow_3_df = pd.read_excel(TOW_PATH_3)

## Clean Data

In [4]:
org_df['year'] = 1
org_3_df['year'] = 3
org_more_df['year'] = 3

org_more_df = org_more_df[org_more_df['SPECIES'] == 'Gadus morhua (Atlantic cod)']

In [5]:
spec_df.drop('SPECIES_GROUP_NAME', axis = 1, inplace = True)
spec_3_df.drop('SPECIES_GROUP_NAME', axis = 1, inplace = True)

spec_df.drop('SPECIES_GROUP_ID', axis = 1, inplace = True)
spec_3_df.drop('SPECIES_GROUP_ID', axis = 1, inplace = True)

spec_df['year'] = 1
spec_3_df['year'] = 3

In [6]:
tow_df['year'] = 1
tow_3_df['year'] = 3

In [7]:
org_df_final = org_df.append(org_3_df).append(org_more_df)
spec_df_final = spec_df.append(spec_3_df)
tow_df_final = tow_df.append(tow_3_df)

In [8]:
org_df_final.columns = [col.lower() for col in org_df.columns]
spec_df_final.columns = [col.lower() for col in spec_df.columns]
tow_df_final.columns = [col.lower() for col in tow_df.columns]

In [9]:
tow_df_final.head()

,operation_id,operation_status,cruise_id,site_name,start_time,start_lat,start_lon,end_time,end_lat,end_lon,year
0,2,Representative,201682,1-52 ALT,6/10/2016 10:54:51 PM,4333.933N,06958.789W,6/10/2016 11:15:11 PM,4333.234N,06959.755W,1
1,3,Representative,201680,8-32,4/18/2016 11:11:17 AM,4212.849N,07019.059W,4/18/2016 11:41:24 AM,4214.227N,07018.776W,1
2,3,Representative,201681,7-29,5/9/2016 9:15:05 AM,4212.533N,07035.072W,5/9/2016 9:45:04 AM,4211.655N,07033.639W,1
3,3,Representative,201682,3-8,6/11/2016 10:55:19 AM,4326.682N,06932.646W,6/11/2016 11:25:19 AM,4325.466N,06933.877W,1
4,4,Representative,201680,8-34,4/18/2016 1:44:38 PM,4209.141N,07003.976W,4/18/2016 2:15:03 PM,4207.861N,07003.031W,1


In [10]:
MERGE_COLS = ['year', 'operation_id', 'cruise_id']

test = pd.merge(tow_df_final, org_df_final, how = 'inner',
         left_on = MERGE_COLS, right_on = MERGE_COLS)

In [11]:
len(tow_df_final.index)

850

In [12]:
spec_df_final.head()

,tow_wt,cruise_id,operation_id,year
0,0.022,201680,53,1
1,0.053,201780,4,1
2,0.072,201781,139,1
3,0.130,201780,156,1
4,0.148,201681,58,1


## Final DFs

In [13]:
catch_df = pd.read_csv('../data/catch_data/combined_cod_catch_data.csv')

In [14]:
catch_df = catch_df.drop(['OPERATION_ID.x', 'OPERATION_ID.y'], axis = 1)
catch_df = catch_df.drop(['CRUISE_ID.x', 'CRUISE_ID.y'], axis = 1)
catch_df = catch_df.drop([col for col in catch_df.columns if '.x' in col], axis = 1)
catch_df.columns = [col.replace('.y', '').lower() for col in catch_df.columns]
catch_df['organism_id'] = catch_df['organism_id'].str.replace(',', '').astype(int)
catch_df['start_time'] = pd.to_datetime(catch_df['start_time'])
catch_df['end_time'] = pd.to_datetime(catch_df['end_time'])

USELESS_COLS = [col for col in catch_df.columns if catch_df[col].nunique() == 1]
catch_df.drop(USELESS_COLS, axis = 1, inplace = True)
catch_df.drop(['start_lat', 'start_lon', 'end_lat', 'end_lon'], axis = 1, inplace = True)

In [15]:
catch_df.to_csv('combined_cod_catch_data_cleaned.csv')